In [20]:
try:
    from sympy import *
    import numpy as np
except:
    !pip install sympy
    

In [21]:
def func1(pai_m, pai_r, method='manu'):
    '''

    :param pai_m: 制造商的利润函数，可以为None
    :param pai_r:快递企业利润函数
    :param method:计算方法，快递企业主导'expr'，制造商主导'manu'，以及集中决策'con'三种方法,默认为制造商主导
    :return:
    '''

    if method == 'manu':
        # 制造商主导,企业利润函数p,pr,l求导
        diff_P_1 = diff(pai_r, P)
        diff_Pr_1 = simplify(diff(pai_r, Pr))
        diff_l_1 = simplify(diff(pai_r, l))
        print("销售价格P一阶导：\t", diff_P_1)
        print("-" * 50)
        print("Pr一阶导:\t", diff_Pr_1)
        print("-" * 50)
        print("l一阶导:\t", diff_l_1)
        print("-" * 50)
        print("销售价格P二阶导：\t", diff(pai_r, P, 2))
        print("-" * 50)
        # 销售价格p
        p1 = simplify(solve(diff_P_1, P)[0])
        print('唯一销售价格p:', p1)
        print("-" * 50)

        # 海瑟矩阵
        arr1 = zeros(2, 2)
        arr1[0, 0] = diff(pai_r, Pr, Pr)
        arr1[0, 1] = diff(pai_r, Pr, l)
        arr1[1, 0] = diff(pai_r, l, Pr)
        arr1[1, 1] = diff(pai_r, l, l)
        A = simplify(arr1.det())
        print("海瑟矩阵A:", A)
        print("-" * 50)

        # 联立方程
        f1 = solve([diff_Pr_1, diff_l_1], [Pr, l])
        pr1 = simplify(f1[Pr])
        l1 = simplify(f1[l])
        print("联立方程")
        print("Pr:\t", pr1)
        print("-" * 50)
        print("l:\t", l1)
        print("-" * 50)

        # 将 p ， rp ，l的表达式带入制造商利润函数
        pai_m = pai_m.subs(P, p1)
        pai_m = pai_m.subs(Pr, pr1)
        pai_m = simplify(pai_m.subs(l, l1))
        print("制造商利润函数:", pai_m)
        print("-" * 50)

        # 分别对Pa，Pm求一阶导
        diff_PA_1 = simplify(diff(pai_m, PA))
        diff_Pm_1 = simplify(diff(pai_m, Pm))
        print('PA一阶导:\t', diff_PA_1)
        print("-" * 50)
        print('Pm一阶导:\t', diff_Pm_1)
        print("-" * 50)
        print('PA二阶导:\t', simplify(diff(pai_m, PA, 2)))
        print("-" * 50)
        print('Pm二阶导:\t', simplify(diff(pai_m, Pm, 2)))
        print("-" * 50)
        print('PA、Pm二阶导:\t', simplify(diff(pai_m, PA, Pm)))
        print("-" * 50)

        B = simplify(diff(pai_m, Pm, 2)) * simplify(diff(pai_m, PA, 2)) - simplify(diff(pai_m, PA, Pm)) * simplify(
            diff(pai_m, PA, Pm))
        B = simplify(B)
        print("海瑟矩阵B:\t", B)
        print("-" * 50)

        # 制造商利润取得最大值,联立求解
        df = solve([diff_PA_1, diff_Pm_1], [PA, Pm])
        pa1 = simplify(df[PA])
        pm1 = simplify(df[Pm])
        print('Pa的解:\t', pa1)
        print("Pm的解:\t", pm1)
        print("-" * 50)

        # 将PA,Pm代入，求得最优销售价格，最优回收价格以及最优服务水平
        p1_ = simplify(p1.subs(PA, pa1))
        pr1_ = pr1.subs(PA, pa1)
        pr1_ = simplify(pr1_.subs(Pm, pm1))
        l1_ = l1.subs(PA, pa1)
        l1_ = simplify(l1_.subs(Pm, pm1))
        print('最优销售价格p:\t', p1_)
        print('最优回收价格pr:\t', pr1_)
        print("最优服务水平l:\t", l1_)
        print("-" * 50)

        # 最优回收量和最优需求量
        qr_ = qr.subs(Pr, pr1_)
        qr_ = simplify(qr_.subs(l, l1_))
        d1_ = simplify(D.subs(P, p1_))
        print("最优回收量qr:\t", qr_)
        print("最优需求量d:\t", d1_)
        print("-" * 50)

        # 制造商和快递企业的利润表达式
        pai_m_ = pai_m.subs(PA, pa1)
        pai_m_ = pai_m_.subs(D, d1_)
        pai_m_ = pai_m_.subs(Pm, pm1)
        pai_m_ = simplify(pai_m_.subs(qr, qr_))

        pai_r_ = pai_r.subs(P, p1_)
        pai_r_ = pai_r_.subs(D, d1_)
        pai_r_ = pai_r_.subs(PA, pa1)
        pai_r_ = pai_r_.subs(qr, qr_)
        pai_r_ = pai_r_.subs(Pr, pr1_)
        pai_r_ = simplify(pai_r_.subs(l, l1_))

        print("制造商利润:\t", pai_m_)
        print("-" * 50)
        print("快递企业利润:\t", pai_r_)
        print("-" * 50)
        print("总利润为：", (pai_m_ + pai_r_))
        # 海瑟矩阵A,B,最优销售价格p,最优回收价格pr,最优服务水平l,最优回收量qr,最优需求量d,制造商给予快递公司的纸箱批发价格PA,Pm,制造商利润,快递企业利润
        return A, B, p1_, pr1_, l1_, qr_, d1_, pa1, pm1, pai_m_, pai_r_

    elif method == 'con':
        '''集中决策方法'''
        # 对决策利润函数p,pr,l求导
        if pai_m != pai_r:
            print("输入相同的决策利润函数")
        else:
            diff_P_1 = diff(pai_r, P)
            diff_Pr_1 = simplify(diff(pai_r, Pr))
            diff_l_1 = simplify(diff(pai_r, l))
            print("销售价格P一阶导：\t", diff_P_1)
            print("-" * 50)
            print("Pr一阶导:\t", diff_Pr_1)
            print("-" * 50)
            print("l一阶导:\t", diff_l_1)
            print("-" * 50)
            print("销售价格P二阶导：\t", diff(pai_r, P, 2))
            print("-" * 50)
            # 销售价格p
            p1 = simplify(solve(diff_P_1, P)[0])
            print('唯一销售价格p:', p1)
            print("-" * 50)

            A = diff(pai_r, Pr, 2) * diff(pai_r, l, 2) - diff(pai_r, Pr, l) * diff(pai_r, Pr, l)
            A = simplify(A)
            print("海瑟矩阵A:", A)
            print("-" * 50)

            f1 = solve([diff_Pr_1, diff_l_1], [Pr, l])
            pr1 = simplify(f1[Pr])
            l1 = simplify(f1[l])
            print("联立方程")
            print("Pr:\t", pr1)
            print("-" * 50)
            print("l:\t", l1)
            print("-" * 50)

            qr_ = qr.subs(Pr, pr1)
            qr_ = simplify(qr_.subs(l, l1))
            d_ = D.subs(P, p1)
            print("最优回收量:\t", qr_)
            print("最优需求量:\t", d_)

            pai_r_ = pai_r.subs(P, p1)
            pai_r_ = pai_r_.subs(D, d_)
            pai_r_ = pai_r_.subs(qr, qr_)
            pai_r_ = pai_r_.subs(Pr, pr1)
            pai_r_ = simplify(pai_r_.subs(l, l1))
            print("集中决策最优利润:\t", pai_r_)
            # 海瑟矩阵A,最优销售价格p,最优回收价格pr,最优服务水平l,最优回收量qr,最优需求量d,集中决策最优利润
            return A, p1, pr1, l1, qr_, d_, pai_r_
    else:
        '''快递企业主导'''
        diff_P_1 = diff(pai_r, P)
         # 销售价格p
        p1 = simplify(solve(diff_P_1, P)[0])
        
        
        pai_m = pai_m.subs(P,p1)
        
        diff_PA_1 = diff(pai_m,PA)
        pa1 = simplify(solve(diff_PA_1,PA)[0])
        print('批发价格PA:',pa1)
        print("-" * 50)
        p1 = p1.subs(PA,pa1)
        print('唯一销售价格p:', p1)
        print("-" * 50)
        
        #Pm一阶导
        diff_Pm_1 = diff(pai_m,Pm)
        print('Pm一阶导',diff_Pm_1)
        print("-" * 50)
        print('Pm二阶导',diff(pai_m,Pm,2))
        print("-" * 50)
        
        pm1 = simplify(solve(diff_Pm_1,Pm)[0])
        print("Pm:",pm1)
        print("-" * 50)
        
        pai_r = pai_r.subs(Pm,pm1)
        pai_r = pai_r.subs(P,p1)
        pai_r = pai_r.subs(PA,pa1)
        print(pai_r)
        
        #分别对r，l求偏导
        diff_r_1 = diff(pai_r,r)
        diff_l_1 = diff(pai_r,l)
        
        #r，l二阶导
        diff_r_2 = diff(pai_r,r,2)
        diff_l_2 = diff(pai_r,l,2)
        print('r二阶导:',diff_r_2)
        print("-" * 50)
        print('l二阶导:',diff_l_2)
        print("-" * 50)
        diff_r_l = diff(pai_r,r,l)
        print('r,l偏导：',diff_r_l)
        print("-" * 50)
        A = diff_r_2*diff_l_2-diff_r_l*diff_r_l
        print('海瑟矩阵A:',simplify(A))
        print("-" * 50)
        
        #联立方程
        df = solve([diff_r_1,diff_l_1],[r,l])
        r1 = simplify(df[r])
        l1 = simplify(df[l])
        print('r:\t',r1)
        print("服务水平l:\t", l1)
        print("-" * 50)
        
        pm1 = pm1.subs(r,r1)
        pm1_ = pm1.subs(l,l1)
        print("最优Pm:",simplify(pm1_))
        print("-" * 50)
        
        pr = Pr.subs(Pm,pm1)
        pr = pr.subs(r,r1)
        
        print('Pr:',simplify(pr))
        print("-" * 50)
        
        qr_ = qr.subs(l,l1)
        qr_ = qr_.subs(Pr,pr)
        print('qr:',simplify(qr_))
        print("-" * 50)
        
        pai_r = pai_r.subs(P,p1)
        pai_r = pai_r.subs(PA,pa1)
        pai_r = pai_r.subs(Pm,pm1_)
        pai_r = pai_r.subs(l,l1)
        pai_r = pai_r.subs(r,r1)
        
        pai_m = pai_m.subs(P,p1)
        pai_m = pai_m.subs(PA,pa1)
        pai_m = pai_m.subs(Pm,pm1_)
        pai_m = pai_m.subs(l,l1)
        pai_m = pai_m.subs(r,r1)
        
        print('快递企业利润：',simplify(pai_r))
        print("-" * 50)
        print('纸箱制造商利润：',simplify(pai_m))
        print("-" * 50)
        print('总利润：',simplify(pai_r+pai_m))
        
        return pa1,p1,l1,pm1_,pr,qr_,pai_r,pai_m

In [22]:
def caculates(values:list):
    values = list(values)
    cm=float(input("新材料制造成本:"))
    cn = float(input("再制造的制造成本:"))
    num = float(input('可直接重复使用的比例:'))
    ct = float(input('政府单位补贴价格:'))
    cd = float(input('运输成本:'))
    cr = float(input('快递企业处理纸箱的单位处理成本:'))
    value_z = float(input('敏感系数:'))
    value_b = float(input('b:'))
    value_a = float(input('a:'))
    value_alpha = float(input('alpha:'))
    value_beta = float(input('beta:'))
    value_k = float(input('k:'))
    value_theta = float(input('theta:'))
    for i in range(len(values)):
        result = values[i].evalf(subs={Cm:cm,Cn:cn,n:num,Ct:ct,Cd:cd,Cr:cr,z:value_z,
                                      a:value_a,b:value_b,alph:value_alpha,beta:value_beta,k:value_k,theta:value_theta})
        print('第{}个数据结果为{}'.format(i+1,round(result,2)))
        
   
    

In [23]:
#论文复现(逆向决策)
Cl, qr = symbols('Cl d', cls=Function)
a, b, k, theta, alph, beta, Cm, Cn, n, Ct, Ct, Pc, Cr, z, Cd = symbols(
    'a b k theta alph beta Cm Cn n t Ct Pc Cr z Cd', real=True, constant=True)
PA, Pm, P, Pr, l = symbols('PA Pm P Pr l', real=True)
prof_m, prof_r, Qr, D = symbols('prof_m prof_r Qr,D', cls=Function)
D = alph - beta * P
Cl = 0.5 * k * l ** 2
qr = theta + a * l + b * Pr
pai_m = (PA - Cm) * (D - qr)+(PA - Cn - Pm) * (1 - n) * qr
pai_r = P * D - PA * (D - n * qr) + Pm * (1 - n) * qr - Pr * qr - Cr * qr - Cl
A, B, p1, pr1, l1, qr_, d1_, pa1, pm1, pai_m_, pai_r_= func1(pai_m, pai_r)

销售价格P一阶导：	 -2*P*beta + PA*beta + alph
--------------------------------------------------
Pr一阶导:	 -Cr*b + PA*b*n - Pm*b*(n - 1) - 2*Pr*b - a*l - theta
--------------------------------------------------
l一阶导:	 -Cr*a + PA*a*n - Pm*a*(n - 1) - Pr*a - 1.0*k*l
--------------------------------------------------
销售价格P二阶导：	 -2*beta
--------------------------------------------------
唯一销售价格p: (PA*beta + alph)/(2*beta)
--------------------------------------------------
海瑟矩阵A: -a**2 + 2.0*b*k
--------------------------------------------------
联立方程
Pr:	 (-Cr*a**2 + Cr*b*k + PA*a**2*n - PA*b*k*n - Pm*a**2*n + Pm*a**2 + Pm*b*k*n - Pm*b*k + k*theta)/(a**2 - 2.0*b*k)
--------------------------------------------------
l:	 a*(Cr*b - PA*b*n + Pm*b*n - Pm*b - theta)/(a**2 - 2.0*b*k)
--------------------------------------------------
制造商利润函数: ((Cm - PA)*(a**2 - 2.0*b*k)*(2*a**2*(Cr*b - PA*b*n + Pm*b*n - Pm*b - theta) + 2*b*(-Cr*a**2 + Cr*b*k + PA*a**2*n - PA*b*k*n - Pm*a**2*n + Pm*a**2 + Pm*b*k*n - Pm*b*k + 

In [24]:
l1 = list([A, B, p1, pr1, l1, qr_, d1_, pa1, pm1, pai_m_, pai_r_])
caculates(l1)

新材料制造成本: 1
再制造的制造成本: 1
可直接重复使用的比例: 1
政府单位补贴价格: 1
运输成本: 1
快递企业处理纸箱的单位处理成本: 1
敏感系数: 1
b: 1
a: 1
alpha: 1
beta: 1
k: 1
theta: 1


第1个数据结果为1.00000000000000
第2个数据结果为0.0
第3个数据结果为1.00000000000000
第4个数据结果为-1.00000000000000
第5个数据结果为0.50
第6个数据结果为0.50
第7个数据结果为0.0
第8个数据结果为1.00000000000000
第9个数据结果为9.08548405369509E+133
第10个数据结果为0.25
第11个数据结果为0.38


In [25]:
#论文复现(正向决策)快递公司主导，没有n
Cl, qr,Pr = symbols('Cl d Pr', cls=Function)
a, b, k, theta, alph, beta, Cm, Cn, n, Ct, Ct, Pc, Cr, z, Cd = symbols(
    'a b k theta alph beta Cm Cn n t Ct Pc Cr z Cd ', real=True, constant=True)
PA, Pm, P, l,r = symbols('PA Pm P l r', real=True)
prof_m, prof_r, Qr, D = symbols('prof_m prof_r Qr,D', cls=Function)
D = alph - beta * P
Pr = Pm-r
Cl = 0.5 * k * l ** 2
qr = theta + b * l + a * Pr
pai_m = (PA - Cm) * D+(Cm-Cn-Pm+Ct)*qr-Cd*qr
pai_r = (P-PA) * D +(Pm-Pr-Cr+Ct)*qr-Cl-Cd*D
func1(pai_m,pai_r,method='expr')

批发价格PA: (alph - beta*(Cd - Cm))/(2*beta)
--------------------------------------------------
唯一销售价格p: (alph + beta*(Cd + (alph - beta*(Cd - Cm))/(2*beta)))/(2*beta)
--------------------------------------------------
Pm一阶导 -Cd*a - a*(Pm - r) + a*(Cm - Cn + Ct - Pm) - b*l - theta
--------------------------------------------------
Pm二阶导 -2*a
--------------------------------------------------
Pm: (a*(-Cd + Cm - Cn + Ct + r) - b*l - theta)/(2*a)
--------------------------------------------------
-Cd*(alph/2 - beta*(Cd + (alph - beta*(Cd - Cm))/(2*beta))/2) - 0.5*k*l**2 + (alph/2 - beta*(Cd + (alph - beta*(Cd - Cm))/(2*beta))/2)*(-(alph - beta*(Cd - Cm))/(2*beta) + (alph + beta*(Cd + (alph - beta*(Cd - Cm))/(2*beta)))/(2*beta)) + (-Cr + Ct + r)*(a*(-r + (a*(-Cd + Cm - Cn + Ct + r) - b*l - theta)/(2*a)) + b*l + theta)
r二阶导: -a
--------------------------------------------------
l二阶导: -1.0*k
--------------------------------------------------
r,l偏导： b/2
-------------------------------------------

((alph - beta*(Cd - Cm))/(2*beta),
 (alph + beta*(Cd + (alph - beta*(Cd - Cm))/(2*beta)))/(2*beta),
 b*(-Cd*a + Cm*a - Cn*a - Cr*a + 2.0*Ct*a + theta)/(4.0*a*k - b**2),
 (a*(-Cd + Cm - Cn + Ct + (-2.0*Cd*a*k + 2.0*Cm*a*k - 2.0*Cn*a*k + 2.0*Cr*a*k - Cr*b**2 + Ct*b**2 + 2.0*k*theta)/(4.0*a*k - b**2)) - b**2*(-Cd*a + Cm*a - Cn*a - Cr*a + 2.0*Ct*a + theta)/(4.0*a*k - b**2) - theta)/(2*a),
 -(-2.0*Cd*a*k + 2.0*Cm*a*k - 2.0*Cn*a*k + 2.0*Cr*a*k - Cr*b**2 + Ct*b**2 + 2.0*k*theta)/(4.0*a*k - b**2) + (a*(-Cd + Cm - Cn + Ct + (-2.0*Cd*a*k + 2.0*Cm*a*k - 2.0*Cn*a*k + 2.0*Cr*a*k - Cr*b**2 + Ct*b**2 + 2.0*k*theta)/(4.0*a*k - b**2)) - b*l - theta)/(2*a),
 a*(-(-2.0*Cd*a*k + 2.0*Cm*a*k - 2.0*Cn*a*k + 2.0*Cr*a*k - Cr*b**2 + Ct*b**2 + 2.0*k*theta)/(4.0*a*k - b**2) + (a*(-Cd + Cm - Cn + Ct + (-2.0*Cd*a*k + 2.0*Cm*a*k - 2.0*Cn*a*k + 2.0*Cr*a*k - Cr*b**2 + Ct*b**2 + 2.0*k*theta)/(4.0*a*k - b**2)) - b*l - theta)/(2*a)) + b**2*(-Cd*a + Cm*a - Cn*a - Cr*a + 2.0*Ct*a + theta)/(4.0*a*k - b**2) + theta,
 -Cd*(al

In [26]:
# 论文复现(集中决策)邹敏
Cl, qr = symbols('Cl d', cls=Function)
a, b, k, theta, alph, beta, Cm, Cn, n, Ct, Ct, Pc, Cr, z, Cd = symbols(
    'a b k theta alph beta Cm Cn n t Ct Pc Cr z Cd', real=True, constant=True)
PA, Pm, P, Pr, l = symbols('PA Pm P Pr l', real=True)
prof_m, prof_r, Qr, D = symbols('prof_m prof_r Qr,D', cls=Function)
D = alph - beta * P
Cl = 0.5 * k * l ** 2
qr = theta + a * l + b * Pr
pai = (P-Cm)*D+(Cm-Cn-Pr-Cr)*qr-Cl
A,p1,pr1,l1,qr_,d_,pai_r_= func1(pai, pai,'con')
l2 = [A,p1,pr1,l1,qr_,d_,pai_r_]
caculates(l2)

销售价格P一阶导：	 -P*beta + alph - beta*(-Cm + P)
--------------------------------------------------
Pr一阶导:	 -Pr*b - a*l - b*(-Cm + Cn + Cr + Pr) - theta
--------------------------------------------------
l一阶导:	 -a*(-Cm + Cn + Cr + Pr) - 1.0*k*l
--------------------------------------------------
销售价格P二阶导：	 -2*beta
--------------------------------------------------
唯一销售价格p: (Cm*beta + alph)/(2*beta)
--------------------------------------------------
海瑟矩阵A: -a**2 + 2.0*b*k
--------------------------------------------------
联立方程
Pr:	 (Cm*a**2 - Cm*b*k - Cn*a**2 + Cn*b*k - Cr*a**2 + Cr*b*k + k*theta)/(a**2 - 2.0*b*k)
--------------------------------------------------
l:	 a*(-Cm*b + Cn*b + Cr*b - theta)/(a**2 - 2.0*b*k)
--------------------------------------------------
最优回收量:	 1.0*b*k*(-Cm*b + Cn*b + Cr*b - theta)/(1.0*a**2 - 2.0*b*k)
最优需求量:	 -Cm*beta/2 + alph/2
集中决策最优利润:	 (0.25*Cm**2*a**2*beta**2 - 0.5*Cm**2*b**2*beta*k - 0.5*Cm**2*b*beta**2*k + 1.0*Cm*Cn*b**2*beta*k + 1.0*Cm*Cr*b**2*beta*k - 0.

新材料制造成本: 1
再制造的制造成本: 1
可直接重复使用的比例: 1
政府单位补贴价格: 1
运输成本: 1
快递企业处理纸箱的单位处理成本: 1
敏感系数: 1
b: 1
a: 1
alpha: 1
beta: 1
k: 1
theta: 1


第1个数据结果为1.00000000000000
第2个数据结果为1.00000000000000
第3个数据结果为-1.00000000000000
第4个数据结果为0.0
第5个数据结果为0.0
第6个数据结果为0.0
第7个数据结果为0.0


In [27]:
 #your work制造商有n
PA, Pm, P, Pr, l = symbols('PA Pm P Pr l', real=True)
a, b, k, theta, alph, beta, Cm, Cn, n, Ct, Ct, Pc, Cr, z, Cd = symbols(
    'a b k theta alph beta Cm Cn n t Ct Pc Cr z Cd', real=True, constant=True)
prof_m, prof_r, qr, D = symbols('prof_m prof_r Qr,D', cls=Function)
qr = theta + a * Pr + b * l
D = alph - beta * P
prof_m = (PA - Cm) * (D - n * qr - (1 - n) * qr) + (PA - Cn - Pm + Ct) * (1 - n) * qr - Cd * (1 - n) * qr
prof_r = P * D - PA * (D - n * qr) + Pm * (1 - n) * qr - (Pr - Ct+Cr) * qr - (k * l ** 2) * 0.5 - (
            Cd * (D - n * qr))

func1(prof_m ,prof_r)

销售价格P一阶导：	 Cd*beta - 2*P*beta + PA*beta + alph
--------------------------------------------------
Pr一阶导:	 Cd*a*n + PA*a*n - Pm*a*(n - 1) - Pr*a - a*(Cr - Ct + Pr) - b*l - theta
--------------------------------------------------
l一阶导:	 Cd*b*n + PA*b*n - Pm*b*(n - 1) - b*(Cr - Ct + Pr) - 1.0*k*l
--------------------------------------------------
销售价格P二阶导：	 -2*beta
--------------------------------------------------
唯一销售价格p: (alph + beta*(Cd + PA))/(2*beta)
--------------------------------------------------
海瑟矩阵A: 2.0*a*k - b**2
--------------------------------------------------
联立方程
Pr:	 (Cd*a*k*n - Cd*b**2*n - Cr*a*k + Cr*b**2 + Ct*a*k - Ct*b**2 + PA*a*k*n - PA*b**2*n - Pm*a*k*n + Pm*a*k + Pm*b**2*n - Pm*b**2 - k*theta)/(2.0*a*k - b**2)
--------------------------------------------------
l:	 b*(Cd*a*n - Cr*a + Ct*a + PA*a*n - Pm*a*n + Pm*a + theta)/(2.0*a*k - b**2)
--------------------------------------------------
制造商利润函数: (-(Cm - PA)*(2.0*a*k - b**2)*(-2*n*(-a*(-Cd*a*k*n + Cd*b**2*n + C

(2.0*a*k - b**2,
 a**2*beta*k*(16.0*n**2 - 32.0*n + 16.0)/(16.0*a*k - 8.0*b**2),
 (0.75*alph + 0.25*beta*(Cd + Cm))/beta,
 (1.0*Cd*a**2*k*n - 0.5*Cd*a**2*k - 1.0*Cd*a*b**2*n + 0.5*Cd*a*b**2 + 0.5*Cm*a**2*k - 0.5*Cm*a*b**2 + 0.5*Cn*a**2*k*n - 0.5*Cn*a**2*k - 0.5*Cn*a*b**2*n + 0.5*Cn*a*b**2 - 0.5*Cr*a**2*k + 0.5*Cr*a*b**2 - 0.5*Ct*a**2*k*n + 1.0*Ct*a**2*k + 0.5*Ct*a*b**2*n - 1.0*Ct*a*b**2 - 1.5*a*k*theta + 0.5*b**2*theta)/(a*(2.0*a*k - 1.0*b**2)),
 b*(1.0*Cd*a*n - 0.5*Cd*a + 0.5*Cm*a + 0.5*Cn*a*n - 0.5*Cn*a - 0.5*Cr*a - 0.5*Ct*a*n + 1.0*Ct*a + 0.5*theta)/(2.0*a*k - 1.0*b**2),
 a*k*(1.0*Cd*a*n - 0.5*Cd*a + 0.5*Cm*a + 0.5*Cn*a*n - 0.5*Cn*a - 0.5*Cr*a - 0.5*Ct*a*n + 1.0*Ct*a + 0.5*theta)/(2.0*a*k - 1.0*b**2),
 0.25*alph - 0.25*beta*(Cd + Cm),
 -0.5*Cd + 0.5*Cm + 0.5*alph/beta,
 0.5*(-Cd*a*beta*n + Cd*a*beta + Cm*a*beta*n - Cm*a*beta - Cn*a*beta*n + Cn*a*beta - Cr*a*beta + Ct*a*beta*n + a*alph*n + beta*theta)/(a*beta*(n - 1)),
 (4.0*Cd**2*a**2*beta*k*n**2 - 4.0*Cd**2*a**2*beta*k*n + 1.0*Cd**

In [28]:
#集中决策
prof = (P-Cm)*D+(Cm-Cn-Cr-Pr+Ct)*qr-(k * l ** 2) * 0.5
func1(prof,prof,'con')

销售价格P一阶导：	 -P*beta + alph - beta*(-Cm + P)
--------------------------------------------------
Pr一阶导:	 -Pr*a - a*(-Cm + Cn + Cr - Ct + Pr) - b*l - theta
--------------------------------------------------
l一阶导:	 -b*(-Cm + Cn + Cr - Ct + Pr) - 1.0*k*l
--------------------------------------------------
销售价格P二阶导：	 -2*beta
--------------------------------------------------
唯一销售价格p: (Cm*beta + alph)/(2*beta)
--------------------------------------------------
海瑟矩阵A: 2.0*a*k - b**2
--------------------------------------------------
联立方程
Pr:	 (Cm*a*k - Cm*b**2 - Cn*a*k + Cn*b**2 - Cr*a*k + Cr*b**2 + Ct*a*k - Ct*b**2 - k*theta)/(2.0*a*k - b**2)
--------------------------------------------------
l:	 b*(Cm*a - Cn*a - Cr*a + Ct*a + theta)/(2.0*a*k - b**2)
--------------------------------------------------
最优回收量:	 1.0*a*k*(Cm*a - Cn*a - Cr*a + Ct*a + theta)/(2.0*a*k - 1.0*b**2)
最优需求量:	 -Cm*beta/2 + alph/2
集中决策最优利润:	 (a*beta*k*(a*k - 0.5*b**2)**2*(Cm*a - Cn*a - Cr*a + Ct*a + theta)*(-Cm*a*k + Cm*b**2 

(2.0*a*k - b**2,
 (Cm*beta + alph)/(2*beta),
 (Cm*a*k - Cm*b**2 - Cn*a*k + Cn*b**2 - Cr*a*k + Cr*b**2 + Ct*a*k - Ct*b**2 - k*theta)/(2.0*a*k - b**2),
 b*(Cm*a - Cn*a - Cr*a + Ct*a + theta)/(2.0*a*k - b**2),
 1.0*a*k*(Cm*a - Cn*a - Cr*a + Ct*a + theta)/(2.0*a*k - 1.0*b**2),
 -Cm*beta/2 + alph/2,
 (a*beta*k*(a*k - 0.5*b**2)**2*(Cm*a - Cn*a - Cr*a + Ct*a + theta)*(-Cm*a*k + Cm*b**2 + Cn*a*k - Cn*b**2 + Cr*a*k - Cr*b**2 - Ct*a*k + Ct*b**2 + k*theta + (2.0*a*k - b**2)*(Cm - Cn - Cr + Ct)) - 0.125*b**2*beta*k*(2.0*a*k - b**2)*(2.0*a*k - 1.0*b**2)*(Cm*a - Cn*a - Cr*a + Ct*a + theta)**2 + (Cm*beta - alph)**2*(a*k - 0.5*b**2)**2*(2.0*a*k - b**2)*(2.0*a*k - 1.0*b**2)/4)/(beta*(a*k - 0.5*b**2)**2*(2.0*a*k - b**2)*(2.0*a*k - 1.0*b**2)))